***
# Python Alchemy - Volume One
# Chapter 14 - Beyond Lists and Dicts

- 14.1 The Power of the Collections Module
- 14.2 Advanced Iteration with Itertools
- 14.3 Algorithmic Helpers: heapq and bisect
- 14.4 Mini Project

***

## 14.1 The Power of the Collections Module

While Python’s built-in data types are powerful and versatile but they sometimes fall short when a task demands specialized behavior or efficiency. This is where the collections module comes into play.

To illustrate, consider a simple log processing example. Suppose you have a list of user actions and you want to find which actions occur most frequently:

In [1]:
from collections import Counter

actions = ["login", "view", "click", "view", "logout", "click", "click"]
freq = Counter(actions)
print(freq.most_common(2))

[('click', 3), ('view', 2)]


In just a few lines, Counter performs what would otherwise require manual iteration.

Similarly, if you were implementing a realtime task queue, you might use a deque to efficiently add and remove tasks:

In [2]:
from collections import deque

tasks = deque(["task1", "task2", "task3"])
tasks.append("task4")
tasks.popleft() # removes 'task1'
print(tasks)

deque(['task2', 'task3', 'task4'])


#### defaultdict: Smarter Dictionaries

One common pain point with regular dictionaries is the need to explicitly check whether a key exists before assigning or updating its value. The defaultdict from Python’s collections module elegantly eliminates this redundancy by automatically initializing missing keys with a default value. For example

In [3]:
from collections import defaultdict
employees = [
    ('Ivaan', 'IT'),
    ('Laisha', 'HR'),
    ('Charlie', 'IT'),
    ('Diana', 'Finance'),
    ('Eve', 'HR')
]

groups = defaultdict(list)
for name, dept in employees:
    groups[dept].append(name)

print(groups)

defaultdict(<class 'list'>, {'IT': ['Ivaan', 'Charlie'], 'HR': ['Laisha', 'Eve'], 'Finance': ['Diana']})


In this example, each department automatically gets an empty list when first encountered, and names are appended without any conditional logic.

The flexibility of defaultdict extends beyond list. Using int as the factory turns it into a counter-like structure

In [4]:
word_count = defaultdict(int)
for word in ["apple", "banana", "apple", "cherry", "banana", "apple"]:
    word_count[word] += 1

print(word_count)

defaultdict(<class 'int'>, {'apple': 3, 'banana': 2, 'cherry': 1})


Similarly, setting the factory to set helps when you need to store unique items per key

In [5]:
activity_log = [
    ('Ivaan', 'login'),
    ('Laisha', 'view'),
    ('Ivaan', 'view'),
    ('Ivaan', 'logout'),
    ('Laisha', 'login')
]
user_actions = defaultdict(set)
for user, action in activity_log:
    user_actions[user].add(action)

print(user_actions)

defaultdict(<class 'set'>, {'Ivaan': {'logout', 'view', 'login'}, 'Laisha': {'view', 'login'}})


#### deque: Fast Queues and Stacks

The deque is implemented as a doubly-linked list under the hood, allowing elements to be efficiently added or removed without costly memory shifts.

Consider a basic example where we simulate a simple task scheduler using a deque:

In [6]:
from collections import deque

tasks = deque(["task1", "task2", "task3"])
tasks.append("task4") # add a task at the end
tasks.appendleft("urgent") # add a high-priority task at the front

print("Initial queue:", tasks)
tasks.popleft() # process the urgent task first
tasks.pop() # remove the last task
print("Remaining queue:", tasks)

Initial queue: deque(['urgent', 'task1', 'task2', 'task3', 'task4'])
Remaining queue: deque(['task1', 'task2', 'task3'])


Deques also shine in algorithmic applications like Breadth-First Search (BFS), where nodes must be processed in the order they were discovered.

In [7]:
from collections import deque

def bfs(graph, start):
    visited = set()
    queue = deque([start])
    while queue:
        node = queue.popleft()
        if node not in visited:
            print(node, end= " ")
            visited.add(node)
            queue.extend(graph[node])

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [], 'E': [], 'F': []
}

bfs(graph, 'A')

A B C D E F 

Another common use case is caching or fixed-size buffers, where the deque’s maxlen parameter ensures automatic removal of the oldest items when new ones are added.

In [8]:
from collections import deque

cache = deque(maxlen=3)
for i in range(1, 6):
    cache.append(i)
    print(list(cache))

[1]
[1, 2]
[1, 2, 3]
[2, 3, 4]
[3, 4, 5]


#### OrderedDict: Keeping Track of Order

Python dictionaries were purely unordered collections. To solve this, Python introduced OrderedDict in the collections module, It is a specialized dictionary subclass that remembers the insertion order of its items, this means every key maintains its position, and iteration respects that sequence.

Consider a simple example where we simulate an access-based cache using OrderedDict:

In [9]:
from collections import OrderedDict

cache = OrderedDict()
cache['A'] = 'Apple'
cache['B'] = 'Banana'
cache['C'] = 'Cherry'

# Access 'A' and move it to the end (most recently used)
cache.move_to_end('A')

# Add a new item and remove the oldest one (least recently used)
cache['D'] = 'Date'
cache.popitem(last=False) # removes 'B'
print(cache)

OrderedDict({'C': 'Cherry', 'A': 'Apple', 'D': 'Date'})


Beyond caching, OrderedDict also plays a vital role in serialization and data transformation, where the sequence of elements can affect meaning.

Here’s a small example demonstrating its use in structured serialization:

In [10]:
from collections import OrderedDict
import json

settings = OrderedDict()
settings['version'] = 1.0
settings['name'] = 'DataProcessor'
settings['priority'] = ['load', 'transform', 'save']
json_data = json.dumps(settings, indent=2)
print(json_data)

{
  "version": 1.0,
  "name": "DataProcessor",
  "priority": [
    "load",
    "transform",
    "save"
  ]
}


The order in which these keys appear is preserved exactly as defined. This feature is crucial when human readability or deterministic outputs are desired.

#### ChainMap: Managing Multiple Contexts

In real-world programming, it’s common to work with data that doesn’t reside in a single source but rather across multiple layers, for example system defaults, environment variables, user settings, or session overrides.

Instead of creating new merged copies using dict.update() or unpacking operators, ChainMap provides a live, dynamic mapping across several dictionaries.

For example, consider a configuration system that combines default settings, environment-specific overrides, and user-provided customizations:

In [11]:
from collections import ChainMap

defaults = {'theme': 'light', 'language': 'en', 'autosave': True}
environment = {'language': 'sv'}
user_settings = {'theme': 'dark'}
config = ChainMap(user_settings, environment, defaults)
print(config['theme']) # dark (user override)
print(config['language']) # sv (environment override)
print(config['autosave']) # True (from defaults)

dark
sv
True


Here, ChainMap gives the illusion of a single cohesive configuration dictionary.

Another key advantage of ChainMap is its flexibility in managing contextual scopes

In [12]:
from collections import ChainMap
global_scope = {'x': 10, 'y': 20}
local_scope = {'y': 99, 'z': 5}
scoped_vars = ChainMap(local_scope, global_scope)
print(scoped_vars['y']) # 99 (local overrides global)
print(scoped_vars['x']) # 10 (from global)

99
10


Moreover, since ChainMap doesn’t merge data into a new structure, it’s incredibly memory-efficient for large configurations or contexts that frequently change. For example:

In [13]:
from collections import ChainMap
# Base configuration (global defaults)
defaults = {
    "host": "localhost",
    "port": 5432,
    "debug": False
}

config = ChainMap(defaults)
# Create a child configuration for a specific environment
dev_config = config.new_child({
    "debug": True,
    "port": 5433
})

print(dev_config["host"]) # localhost
print(dev_config["port"]) # 5433
print(dev_config["debug"]) # True

localhost
5433
True


## Advanced Iteration with Itertools

Lazy iteration reflects a broader design philosophy in Python that favors computing values only when they are actually needed, rather than generating everything in advance.

The itertools module provides a toolkit of fast, memory-efficient functions for building such pipelines. It includes tools that fall into three main categories:
- Infinite iterators
- Combinatoric iterators
- Terminating iterators 

#### itertools.accumulate()

In [14]:
import itertools

data = [5, 10, 15, 20]
cumulative = itertools.accumulate(data)
print(list(cumulative))

[5, 15, 30, 50]


Here, accumulate() generates running totals without ever storing additional lists.

Similarly, for large data streams such as server logs, you might want to process entries line by line without reading the entire file into memory.

In [15]:
import itertools

with open("sample\\server.log") as logs:
    first_10 = itertools.islice(logs, 10)
    for line in first_10:
        print(line.strip())

2025-08-01 09:00:01 INFO  Server started on port 8080
2025-08-01 09:00:05 INFO  User login successful user_id=1023
2025-08-01 09:00:09 WARN  High memory usage detected (78%)
2025-08-01 09:00:12 INFO  GET /api/status 200
2025-08-01 09:00:15 INFO  POST /api/login 201
2025-08-01 09:00:18 ERROR Database connection timeout
2025-08-01 09:00:22 INFO  Retrying database connection
2025-08-01 09:00:25 INFO  Database connection restored
2025-08-01 09:00:28 INFO  GET /api/users 200
2025-08-01 09:00:31 INFO  User logout user_id=1023


This example prints only the first ten lines from a potentially massive log file.

#### Infinite Iterators

Infinite iterators are the generators that never stop unless you explicitly tell them to.

In [16]:
import itertools
for i in itertools.count(start=10, step=5):
    if i > 30:
        break
    print(i)

10
15
20
25
30


Likewise, cycle() is ideal when you want to repeat a pattern indefinitely

In [17]:
import itertools

colors = ['red', 'green', 'blue']
for color, _ in zip(itertools.cycle(colors), range(6)):
    print(color)

red
green
blue
red
green
blue


Similarly, repeat() can be used to provide constant values

In [18]:
import itertools
for msg in itertools.repeat("Processing...", 3):
    print(msg)

Processing...
Processing...
Processing...


#### Terminating Iterators

Terminating iterators, by contrast, operate exclusively over finite data domains and yield computed results derived from pre-existing sequences.

#### itertools.accumulate

Accumulate creates an iterator that produces running results

In [19]:
import itertools
values = [2, 4, 6, 8]
print(list(itertools.accumulate(values)))

[2, 6, 12, 20]


#### itertools.chain

Chain constructs an iterator that traverses multiple input iterables as if they were a single continuous sequence.

In [ ]:
import itertools

a = [1, 2]
b = [3, 4, 5]
c = [6]
combined = itertools.chain(a, b, c)
print(list(combined))

[1, 2, 3, 4, 5, 6]


#### itertools.compress

Compress filters elements from one iterable using a corresponding sequence of Boolean selectors.

In [ ]:
import itertools

data = ['apple', 'banana', 'cherry', 'date']
selectors = [1, 0, 1, 0]
print(list(itertools.compress(data, selectors)))

['apple', 'cherry']


#### itertools.islice

islice provides fine-grained slicing for any iterator without converting it into a list.

In [22]:
import itertools

numbers = itertools.count(10)
print(list(itertools.islice(numbers, 5)))

[10, 11, 12, 13, 14]


#### Combinatoric Iterators

From generating test cases to exploring search spaces or simulating different outcomes, combinatorics plays a central role in many computational problems.

#### product() — Cartesian Product

The product() function computes the Cartesian product of input iterables, generating tuples that represent all possible ordered pairs (or n-tuples).

In [23]:
import itertools

letters = ['A', 'B']
numbers = [1, 2, 3]
for pair in itertools.product(letters, numbers):
    print(pair)

('A', 1)
('A', 2)
('A', 3)
('B', 1)
('B', 2)
('B', 3)


Here, product() produces every combination of a letter followed by a number.

#### permutations() — Ordered Arrangements

The permutations() iterator produces all possible ordered arrangements of a given iterable.

In [24]:
import itertools
for p in itertools.permutations(['red', 'green', 'blue'], 2):
    print(p)

('red', 'green')
('red', 'blue')
('green', 'red')
('green', 'blue')
('blue', 'red')
('blue', 'green')


#### combinations() — Unordered Selections

While permutations() focuses on order, combinations() deals with unordered selections

In [25]:
import itertools
for c in itertools.combinations(['A', 'B', 'C', 'D'], 2):
    print(c)

('A', 'B')
('A', 'C')
('A', 'D')
('B', 'C')
('B', 'D')
('C', 'D')


#### combinations_with_replacement() — Allowing Repetition

Finally, combinations_with_replacement() extends combinations by allowing elements to repeat within a group.

In [26]:
import itertools
for c in itertools.combinations_with_replacement(['x', 'y', 'z'], 2):
    print(c)

('x', 'x')
('x', 'y')
('x', 'z')
('y', 'y')
('y', 'z')
('z', 'z')


## 14.3 Algorithmic Helpers: heapq and bisect

Both heapq and bisect are built for working efficiently with ordered data.

#### heapq: Priority Queues and Efficient Retrieval

A heap is a specialized tree-based structure that maintains the smallest (or largest) element at the root, allowing rapid access to priority items.

For example, consider an application managing incoming tasks with varying priorities.

In [27]:
import heapq

tasks = []
heapq.heappush(tasks, (2, 'Update Database'))
heapq.heappush(tasks, (1, 'Process Payment'))
heapq.heappush(tasks, (3, 'Send Notification'))

# Retrieve task with highest priority (lowest number)
priority, task = heapq.heappop(tasks)
print(f"Next task: {task} (priority {priority})")

Next task: Process Payment (priority 1)


#### bisect: Maintaining Sorted Order

The bisect module complements heapq by providing fast, index-based search and insertion into sorted lists.

Here’s a simple yet powerful example: maintaining a sorted list of student scores as new results arrive.

In [28]:
import bisect

scores = [45, 67, 78, 89]
new_score = 73

bisect.insort(scores, new_score)
print(scores)

[45, 67, 73, 78, 89]


The companion function bisect.bisect() finds the exact position where an element should be inserted:

In [29]:
position = bisect.bisect(scores, 80)
print(f"Insert position for 80: {position}")

Insert position for 80: 4
